# MMA基金資料

In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [2]:
import re
from collections import defaultdict

______

## 國內

### 基本資料


In [60]:
uid = 'ACUS03-UN2'
url_basic = 'http://mmafund.sinopac.com/w/wr/wr01.djhtm?a=ACUS03-UN2'
res = requests.get(url_basic)

In [35]:
soup = BS(res.text)

C:\Anaconda2\envs\py3k\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\py3k\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


* 基本資料

In [61]:
fieldSoup = soup.select('.wfb5l')
fieldList = [uid]
for e in fieldSoup[:10]:
#     print(e.text)
    fieldList.append(e.text)
starNum = getFundStar(fieldSoup[-3])
fieldList.append(starNum)
fieldList.append(fieldSoup[-1].text)
fieldList

星星幾顆:4.5


['ACUS03-UN2',
 '永豐領航科技基金',
 '永豐投信',
 '1998年9月4日',
 '楊明昌',
 '6.43台幣 (2017/01/31)',
 '2.00(台幣)',
 '國內股票開放型科技類',
 '中國信託銀行',
 '台灣',
 '台灣',
 4.5,
 '本基金以投資從事或轉投資於通訊、資訊、半導體、關鍵性零組件、生物工程、消費性電子、精密機械與自動化、航太、高級材料、特用化學與製藥、醫療保健、污染防治、資源開發、高級感測及其他經財政部會同主管機關認定重要科技之上市、上櫃股票為主，投資總額不低於淨資產之60%']

In [62]:
info_colName = [
    'uid',
    '基金名稱',
    '基金公司',
    '成立日期',
    '基金經理人',
    '基金規模(億)',
    '成立時規模(億)',
    '基金類型',
    '保管銀行',
    '主要投資區域',
    '投資區域',
    '基金評等',
    '投資標的'
]
list(zip(info_colName,fieldList))

[('uid', 'ACUS03-UN2'),
 ('基金名稱', '永豐領航科技基金'),
 ('基金公司', '永豐投信'),
 ('成立日期', '1998年9月4日'),
 ('基金經理人', '楊明昌'),
 ('基金規模(億)', '6.43台幣 (2017/01/31)'),
 ('成立時規模(億)', '2.00(台幣)'),
 ('基金類型', '國內股票開放型科技類'),
 ('保管銀行', '中國信託銀行'),
 ('主要投資區域', '台灣'),
 ('投資區域', '台灣'),
 ('基金評等', 4.5),
 ('投資標的',
  '本基金以投資從事或轉投資於通訊、資訊、半導體、關鍵性零組件、生物工程、消費性電子、精密機械與自動化、航太、高級材料、特用化學與製藥、醫療保健、污染防治、資源開發、高級感測及其他經財政部會同主管機關認定重要科技之上市、上櫃股票為主，投資總額不低於淨資產之60%')]

* 找星星幾枚?

In [44]:
starList = fieldSoup[-3].select('img') # bs4 tag
starList = [star['src'] for star in starList ] # 取出src裡面的url
starStr = '**'.join(starList) # 接成string
# starList[0]['src']
if re.findall('/BT_Coin_b.gif',starStr):
    starNum = 0.5
else:
    starNum = 0
starNum += len(re.findall('/BT_Coin_a.gif',starStr))
print('星星幾顆:{}'.format(starNum))
# starStr

星星幾顆:4.5


In [56]:
getFundStar(fieldSoup[-3])

星星幾顆:4.5


4.5

#### 歷任經理人

In [119]:
managerList = []

managerBSTable = soup.select('#oMainTable')[1]


managerList = []
for manager in managerBSTable.select('tr')[2:]:
    row = []
    for index,field in enumerate(manager.select('td')):
        try:
            row.append(float(field.text)) # 轉換數字部分
        except ValueError as e:
            row.append(field.text) 
    print(row)

['楊明昌', '2017/2/3至今', 0.0, 4.06, 3.71, '永豐高科技基金\n永豐領航科技基金']
['林靜怡', '2014/08/01至2017/2/2', 30.0, 27.23, 1.21, '']
['李盈儀', '2014/05/02至2014/7/31', 2.0, 7.63, 5.97, '永豐中小基金\n永豐永豐基金']
['邱鶴倫', '2012/04/01至2014/5/1', 25.0, 46.81, 10.82, '']
['謝其勛', '2011/04/29至2012/3/31', 11.0, -22.13, -12.25, '']
['李維岳', '2010/04/20至2011/4/28', 12.0, 6.0, 15.11, '']
['謝其勛', '2007/11/05至2010/4/19', 29.0, -27.62, -15.3, '']
['張家維', '2003/01/02至2007/11/4', 58.0, 201.6, 108.27, '']
['謝宗權', '2002/03/20至2003/1/1', 10.0, -41.2, -26.52, '']
['葉順吉', '2001/03/01至2002/3/19', 12.0, 5.08, 7.4, '']
['林大鈞', '1998/09/01至2001/2/28', 29.0, 21.6, -16.56, '富蘭克林華美積極回報債券組合基金-累積型(台幣)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美積極回報債券組合基金-分配型(台幣)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美積極回報債券組合基金-累積型(美元)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美積極回報債券組合基金-分配型(人民幣)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美全球投資級債券基金-分配型(台幣)(本基金之配息來源可能為本金)\n富蘭克林華美積極回報債券組合基金-分配型(美元)(本基金有相當比重投資於非投資等級之高風險債券且基金之配息來源可能為本金)\n富蘭克林華美全球投資級債券基金-分配型(

### 持股狀況

In [187]:
url_fund = 'http://mmafund.sinopac.com/w/wr/wr04.djhtm?a=ACJS24-Z21'
res =requests.get(url_fund)

In [188]:
soup = BS(res.text)

C:\Anaconda2\envs\py3k\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\py3k\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [189]:
update_dateStr = soup.select('#oMainTable')[1].select('tr')[0].text # 資料月份
update_date = re.findall(r'\d+/\d+',update_dateStr)[0]
print(update_date)

2017/01


In [190]:
for e in soup.select('#oMainTable')[1].select('tr')[1:]:
    row1 = []; row2=[];
    for index,field in enumerate(e.select('td')):
        if index > 3:
            row2.append(field.text)
        else:
            row1.append(field.text)
    print('row1:%s \nrow2:%s' %(row1,row2))

row1:['股票名稱', '持股(千股)', '比例', '增減'] 
row2:['股票名稱', '持股(千股)', '比例', '增減']
row1:['Tencent', 'N/A', '6.65', 'N/A'] 
row2:['KOITO MFG CO', 'N/A', '3.07', '-0.14%']
row1:['大立光', '6', '6.61', '4.36%'] 
row2:['華夏滬深300指數ETF', 'N/A', '2.36', '0.00%']
row1:['台積電', '130', '5.97', '0.05%'] 
row2:['Telekomunikasi Indonesia TBK', 'N/A', '2.25', '-0.15%']
row1:['NIDEC CORP', 'N/A', '3.63', '0.14%'] 
row2:['正新', '145', '2.24', '0.89%']
row1:['環球晶', '90', '3.19', '-3.25%'] 
row2:['iShares 安碩富時A50 中國指數ETF', 'N/A', '2.23', '0.03%']


### 基金持股分布

* 依持有類股
* 依產業
* 依區域

資料藏在js語法裏頭

In [177]:
soup.select('#oMainTable')[2].select('div')

[<div class="wfb1ar">資料日期：2017/02/17</div>,
 <div id="SysJustWebGraphDIV1"></div>]

In [194]:
string1 = 'DJGraphObj1'
html_text = res.text
target_text = html_text[html_text.index(string1):]
# print(html_text[html_text.index(string1):])
print(target_text)

DJGraphObj1;
	var $DJGraphObj2;
	var $DJGraphObj3;
	
	var sRenderTo1 = 'SysJustWebGraphDIV1';
	var sRenderTo2 = 'SysJustWebGraphDIV2';
	var sRenderTo3 = 'SysJustWebGraphDIV3';
	
	$(document).ready(function () {
		if (!!$DJWebGraph) {

			$DJGraphObj1 = new $DJWebGraph();
	        $DJGraphObj2 = new $DJWebGraph();
			$DJGraphObj3 = new $DJWebGraph();
        	
			if (sRenderTo1 != '')
			{
				$DJGraphObj1.CreateGraphObject({
					'CanvasClass': 'pie', //表示為 PIE 圖形
					'ID': 'wb04-1',
					'Width': '500',
					'Height': '300',
					'RenderTo': sRenderTo1,
					'Title': ['', '基金持股分佈-依持有類股[萬元]','亞洲不含日本','日本','大陸地區','上市光電','上市半導體','上櫃半導體','上市橡膠','上市電子零組件','上市油電燃氣','上櫃其他電子','上市電腦及週邊設備','其它'],
					'PieBorder' : '0',
					'BgColor': 'ffffff',
					'PieNOTB': 0,		//0:顯示表格,1:不顯示表格,2:不顯示表格,只顯示名稱
					'PieSplit': ['1','0','0','0','0','0','0','0','0','0','0','0'],
					'PieV': ['16416','4583','3558','2681','2431','1619','937','892','844','440','279','5702'],
					'Pie_AddHeaders': '名稱 投資金額(以萬元為單

In [317]:
pat1 = r"(?:\'Title\':)(.+)"
investTitle = re.findall(pat,target_text)
print(investTitle[0])

 ['', '基金持股分佈-依持有類股[萬元]','亞洲不含日本','日本','大陸地區','上市光電','上市半導體','上櫃半導體','上市橡膠','上市電子零組件','上市油電燃氣','上櫃其他電子','上市電腦及週邊設備','其它'],


In [330]:
pat2 = r"(?:\')(.*?)(?:\')"
investTitleByStock = re.findall(pat2,investTitle[0]) ## 依產業標題
investTitleByStock

['',
 '基金持股分佈-依持有類股[萬元]',
 '亞洲不含日本',
 '日本',
 '大陸地區',
 '上市光電',
 '上市半導體',
 '上櫃半導體',
 '上市橡膠',
 '上市電子零組件',
 '上市油電燃氣',
 '上櫃其他電子',
 '上市電腦及週邊設備',
 '其它']

In [332]:
investTitleByArea = re.findall(pat2,investTitle[1])
investTitleByArea

['', '基金持股分佈-依區域[萬元]', '香港', '台灣', '中國', '日本', '現金', '東南亞', '韓國']

In [315]:
investTitleByIndustry = re.findall(pat2,investTitle[2])
investTitleByIndustry

['',
 '基金持股分佈-依產業[萬元]',
 '科技',
 '工業',
 '消費',
 '金融保險',
 '現金',
 'A股ETF',
 '能源',
 '原物料',
 '公共事業',
 '醫療保健',
 '其他']

In [335]:
pat3 = r"(?:\'PieV\':)(.+)"
investByStock = re.findall(pat2,re.findall(pat3,target_text)[0])
investByArea = re.findall(pat2,re.findall(pat3,target_text)[1])
investByIndustry = re.findall(pat2,re.findall(pat3,target_text)[2])

______

# 輔助函數

In [6]:
def getFundStar(starTag):
    '''取得基金評等數字(影像<星星>標記 --> 數字)
    params ------
    starTag: 含有星號圖案的bs4 tag
    return: 基金評等(int)
    '''
    starList = starTag.select('img') # bs4 tag
    starList = [star['src'] for star in starList ] # 取出src裡面的url
    starStr = '**'.join(starList) # 接成string
    if re.findall('/BT_Coin_b.gif',starStr):
        starNum = 0.5
    else:
        starNum = 0
    starNum += len(re.findall('/BT_Coin_a.gif',starStr))
    print('星星幾顆:{}'.format(starNum))
    return starNum

In [57]:
def getDomesticShareHolding(html_text):
    """取得國內持股資料
    剖析mma中html含有js程式碼，資料隱藏在js其中
    params
    html_text : raw text(str)
    return : defaultdict(list of tuple)
    """
    string1 = 'DJGraphObj1' # 切出目標字串    
    target_text = html_text[html_text.index(string1):]
    
    pat1 = r"(?:\'Title\':)(.+)"
    investTitle = re.findall(pat1,target_text) # 取得並切分表單table
           
    pat2 = r"(?:\')(.*?)(?:\')" # 取出包含在 ' '內的字串
    pat3 = r"(?:\'PieV\':)(.+)" # 取出包含在 PieV 後的字串
#     investTitleByStock = re.findall(pat2,investTitle[0]) ## 依產業標題(List)
#     investTitleByStock
    table = defaultdict(list)
    for index,titleText in enumerate(investTitle):
        titleList = re.findall(pat2,titleText)
        if len(titleList) == 1:
            continue
        colname = titleList[1]
        titleList = titleList[2:]
        valueList = re.findall(pat2,re.findall(pat3,target_text)[index])
#         print(titleList,valueList)
        table[colname]= (list(zip(titleList,valueList)))
    return table

In [116]:
def getDomesticStockHolding(soup):
    """取得國內持股資料中股票張數
    params 
    ======
    input: soup --- html經BS包裝後變數
    return : list of list of tuple [[(股票名稱,xxx),(持股(千股),oooo),(比例,...),(增減,...)]]
    """

    def parseElement(e):
        """利用此函數剖析表格內元素
        """
        row1 = []; row2=[];
        for index,field in enumerate(e.select('td')):        
            if index > 3:
                row2.append(field.text)
            else:
                row1.append(field.text)
        return row1,row2
    
    fundShareTableList = []
    rowData = {}
    for index,e in enumerate(soup.select('#oMainTable')[1].select('tr')[1:]):
        colnames = []    
        if index == 0:
            col1,col2 = parseElement(e)            
        else:            
            row1,row2 = parseElement(e)
#             rowData[col1]=row1
            
            fundShareTableList.append(list(zip(col1,row1))) 
            fundShareTableList.append(list(zip(col2,row2)))
    
    return fundShareTableList

In [117]:
tables = getDomesticStockHolding(soup)

['台積電', '345', '7.77', '-0.50%']
['大立光', '10', '5.40', '0.40%']
['環球晶圓', 'N/A', '4.43', '1.39%']
['華新科', '900', '4.21', '0.01%']
['台半', '949', '4.17', '-0.25%']


In [110]:
list(zip(*tables[0]))
    

[('股票名稱', '持股(千股)', '比例', '增減'), ('台積電', '345', '7.77', '-0.50%')]

In [87]:
for e in soup.select('#oMainTable')[1].select('tr')[1:]:
    row1,row2 = parseElement(e)
    print(row2)

['股票名稱', '持股(千股)', '比例', '增減']
['雙鴻', '326', '3.67', '-0.58%']
['同欣電', '245', '3.52', '-0.04%']
['愛普', '330', '3.27', '0.13%']
['盟立', '680', '3.19', '-0.16%']
['台勝科', '373', '3.13', '-0.07%']


In [356]:
test['基金持股分佈-依區域[萬元]']

[('香港', '10823'),
 ('台灣', '9652'),
 ('中國', '6179'),
 ('日本', '4684'),
 ('現金', '4402'),
 ('東南亞', '4321'),
 ('韓國', '323')]

In [61]:
url = 'http://mmafund.sinopac.com/w/wr/wr04.djhtm?a=ACSH03-170'
res = requests.get(url)
html_text = res.text
soup = BS(html_text)

C:\Anaconda2\envs\py3k\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda2\envs\py3k\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [62]:
getDomesticShareHolding(html_text)

defaultdict(list,
            {'基金持股分佈-依持有類股[萬元]': [('上市半導體', '18374'),
              ('上櫃半導體', '12738'),
              ('上市光電', '10835'),
              ('上市電子零組件', '8001'),
              ('上櫃電子零組件', '6394'),
              ('上市其他', '4573'),
              ('上櫃其他電子', '3213'),
              ('上市其他電子', '2653'),
              ('上市電機機械', '3510'),
              ('上市通信網路', '1425'),
              ('上市建材營造', '1953'),
              ('上櫃通信網路', '1104'),
              ('其它', '7622')]})